Authors: A.Radhakrishnan, GFDL


Search for mlotst in select CMIP models

In [1]:
import xarray as xr
import intake
import intake_esm
import sys
from dask_gateway import Gateway
import pandas as pd
pd.set_option("display.max_colwidth", None)
pd.options.display.max_rows = None

esgf-world.json is the ESM collections spec file for the netCDF data in the S3 bucket esgf-world. 
The catalog is updated on an on-demand basis for now. 
You can refer to https://github.com/aradhakrishnanGFDL/gfdl-aws-analysis/tree/community/esm-collection-spec-examples for the most recent catalogs
More examples can be found in https://github.com/aradhakrishnanGFDL/gfdl-aws-analysis/tree/community/examples 


In [2]:
col_url = "https://cmip6-nc.s3.us-east-2.amazonaws.com/esgf-world.json" 

In [3]:
col = intake.open_esm_datastore(col_url)
esmcol_data = col.esmcol_data

In [4]:
def latest_version(cat):
    """
    input
    cat: esmdatastore
    output
    esmdatastore with latest DRS versions
    """
    latest_cat = cat.df.sort_values(by=['version','path']).drop_duplicates(['temporal subset','model','mip_table',
                                               'institute','variable','ensemble_member',
                                               'grid_label','experiment_id'],keep='last')
    return latest_cat

In [5]:
query_Omon_mlotst = dict(experiment_id=['abrupt-4xCO2','1pctCO2','historical'],
                 mip_table=['Omon'],
                 ensemble_member=["r1i1p1f1","r1i1p1f2"],
                 model=['CESM2','CNRM-CM6-1','UKESM1-0-LL','GFDL-ESM4','IPSL-CM6A-LR'],
                 grid_label=['gn'],
                 variable=["mlotst"])

cat_Omon_mlotst = col.search(**query_Omon_mlotst)
cat_Omon_mlotst_lat = latest_version(cat_Omon_mlotst)


In [10]:
cat_Omon_mlotst_latest = intake.open_esm_datastore(cat_Omon_mlotst_lat,esmcol_data=esmcol_data)

In [54]:
cat_Omon_mlotst_latest.df.groupby(['model']).nunique() #'CNRM-CM6-1','UKESM1-0-LL', missing

,project,institute,experiment_id,frequency,modeling_realm,mip_table,ensemble_member,grid_label,variable,temporal subset,version,path
model,,,,,,,,,,,,
CESM2,1,1,1,1,1,1,1,1,1,1,1,1
GFDL-ESM4,1,1,1,1,1,1,1,1,1,9,1,9
IPSL-CM6A-LR,1,1,1,1,1,1,1,1,1,1,1,1


In [44]:
cat_Omon_mlotst_latest.df.groupby(['variable','model','version','temporal subset']).nunique()

project  institute  \
variable model        version   temporal subset                       
mlotst   CESM2        v20190308 185001-201412          1          1   
         GFDL-ESM4    v20190726 185001-186912          1          1   
                                187001-188912          1          1   
                                189001-190912          1          1   
                                191001-192912          1          1   
                                193001-194912          1          1   
                                195001-196912          1          1   
                                197001-198912          1          1   
                                199001-200912          1          1   
                                201001-201412          1          1   
         IPSL-CM6A-LR v20180803 185001-201412          1          1   

                                                 experiment_id  frequency  \
variable model        version   temporal subset                             
mlotst   CESM2        v20190308 185001-201412                1          1   
         GFDL-ESM4    v20190726 185001-186912                1          1   
                                187001-188912                1          1   
                                189001-190912                1          1   
                                191001-192912                1          1   
                                193001-194912                1          1   
                                195001-196912                1          1   
                                197001-198912                1          1   
                                199001-200912                1          1   
                                201001-201412                1          1   
         IPSL-CM6A-LR v20180803 185001-201412                1          1   

                                                 modeling_realm  mip_table  \
variable model        version   temporal subset                              
mlotst   CESM2        v20190308 185001-201412                 1          1   
         GFDL-ESM4    v20190726 185001-186912                 1          1   
                                187001-188912                 1          1   
                                189001-190912                 1          1   
                                191001-192912                 1          1   
                                193001-194912                 1          1   
                                195001-196912                 1          1   
                                197001-198912                 1          1   
                                199001-200912                 1          1   
                                201001-201412                 1          1   
         IPSL-CM6A-LR v20180803 185001-201412                 1          1   

                                                 ensemble_member  grid_label  \
variable model        version   temporal subset                                
mlotst   CESM2        v20190308 185001-201412                  1           1   
         GFDL-ESM4    v20190726 185001-186912                  1           1   
                                187001-188912                  1           1   
                                189001-190912                  1           1   
                                191001-192912                  1           1   
                                193001-194912                  1           1   
                                195001-196912                  1           1   
                                197001-198912                  1           1   
                                199001-200912                  1           1   
                                201001-201412                  1           1   
         IPSL-CM6A-LR v20180803 185001-201412                  1           1   

                                                 path  
variable mo

In [46]:
dset_dict_mlotst = cat_Omon_mlotst_latest.to_dataset_dict(storage_options=dict(anon=True),cdf_kwargs={'decode_times': False, 'chunks': {}})


--> The keys in the returned dictionary of datasets are constructed as follows:
	'project.institute.model.experiment_id.mip_table'


In [50]:
dset_dict_mlotst['CMIP6.IPSL.IPSL-CM6A-LR.historical.Omon'].mlotst

<xarray.DataArray 'mlotst' (ensemble_member: 1, time: 1980, y: 332, x: 362)>
dask.array<broadcast_to, shape=(1, 1980, 332, 362), dtype=float32, chunksize=(1, 1980, 332, 362), chunktype=numpy.ndarray>
Coordinates:
    nav_lat          (y, x) float32 dask.array<chunksize=(332, 362), meta=np.ndarray>
    nav_lon          (y, x) float32 dask.array<chunksize=(332, 362), meta=np.ndarray>
  * time             (time) float64 15.5 45.0 74.5 ... 6.022e+04 6.025e+04
  * ensemble_member  (ensemble_member) <U8 'r1i1p1f1'
Dimensions without coordinates: y, x
Attributes:
    standard_name:       ocean_mixed_layer_thickness_defined_by_sigma_t
    long_name:           Ocean Mixed Layer Thickness Defined by Sigma T
    units:               m
    online_operation:    average
    cell_methods:        area: mean where sea time: mean
    interval_operation:  2700 s
    interval_write:      1 month
    cell_measures:       area: areacello
    description:         Sigma T is potential density referenced to ocean sur...
    history:             none